In [30]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [31]:
# Function to load and preprocess images
def load_images_and_labels(base_dir, annotations_df):
    images = []
    labels = []
    for _, row in annotations_df.iterrows():
        img_path = os.path.join(base_dir, row['filename'])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))  # Resize for model input
        images.append(img)
        labels.append(row['class'])
    return np.array(images), np.array(labels)

In [32]:
# Paths to dataset folders
dataset_base = 'C:\\Users\\Daesut\\Desktop\\AppliedAI\\Philippine Currencies.v1i.tensorflow'
train_dir = os.path.join(dataset_base, 'train')
valid_dir = os.path.join(dataset_base, 'valid')
test_dir = os.path.join(dataset_base, 'test')

In [33]:
# Load annotations and images
train_annotations_df = pd.read_csv(os.path.join(train_dir, '_annotations.csv'))
valid_annotations_df = pd.read_csv(os.path.join(valid_dir, '_annotations.csv'))
test_annotations_df = pd.read_csv(os.path.join(test_dir, '_annotations.csv'))
train_images, train_labels = load_images_and_labels(train_dir, train_annotations_df)
valid_images, valid_labels = load_images_and_labels(valid_dir, valid_annotations_df)
test_images, test_labels = load_images_and_labels(test_dir, test_annotations_df)

In [34]:
# Label encoding and one-hot encoding
label_encoder = LabelEncoder()
all_labels = np.unique(np.concatenate((train_labels, valid_labels, test_labels)))
label_encoder.fit(all_labels)
train_labels_encoded = label_encoder.transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)
test_labels_encoded = label_encoder.transform(test_labels)
train_labels_one_hot = to_categorical(train_labels_encoded)
valid_labels_one_hot = to_categorical(valid_labels_encoded)
test_labels_one_hot = to_categorical(test_labels_encoded)

In [35]:
# Model setup with fine-tuning and increased dropout for regularization
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Set the whole model as trainable

In [36]:
# Fine-tune from this layer onwards
fine_tune_at = 100  # Fine-tune from the 100th layer onwards
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


In [37]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Increased dropout
predictions = Dense(train_labels_one_hot.shape[1], activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=50,  # More aggressive rotation
    width_shift_range=0.25,  # More shift
    height_shift_range=0.25,  # More shift
    shear_range=0.25,  # More shear
    zoom_range=0.25,  # More zoom
    horizontal_flip=True,
    fill_mode='nearest'
)

In [39]:
# Train the model for more epochs
model.fit(train_datagen.flow(train_images, train_labels_one_hot, batch_size=32),
          epochs=50,  # Increased epochs for more extensive training
          validation_data=(valid_images, valid_labels_one_hot))

Epoch 1/50


c:\Users\Daesut\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.1111 - loss: 3.0386 - val_accuracy: 0.0000e+00 - val_loss: 2.5823
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step - accuracy: 0.0556 - loss: 2.6465 - val_accuracy: 0.0000e+00 - val_loss: 2.6154
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step - accuracy: 0.1111 - loss: 2.3992 - val_accuracy: 0.0000e+00 - val_loss: 2.6686
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step - accuracy: 0.1111 - loss: 2.4685 - val_accuracy: 0.0000e+00 - val_loss: 2.7183
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step - accuracy: 0.3333 - loss: 1.9678 - val_accuracy: 0.0000e+00 - val_loss: 2.7788
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/step - accuracy: 0.1667 - loss: 1.9628 - val_accuracy: 0.0000e+00 - val_loss: 2.8473
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step - accuracy: 0.5556 - loss: 1.9961 - val_accuracy: 0.0000e+00 - val_loss: 2.9190
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step - accuracy: 0.5556 - loss: 1.8203 - val_accuracy: 0.

In [40]:
# Real-time detection setup
cap = cv2.VideoCapture(0)  # Start webcam capture
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Preprocess the frame for model input
    image = cv2.resize(frame, (224, 224))
    image = image.astype('float32') / 255
    image = np.expand_dims(image, axis=0)
    
    # Predict the currency in the frame
    prediction = model.predict(image)
    label = label_encoder.inverse_transform([np.argmax(prediction)])
    
    # Display the predicted label on the frame
    cv2.putText(frame, 'Label: {}'.format(label), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Currency Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 811ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 